## Preprocessing

In [271]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
import warnings
warnings.filterwarnings('ignore')

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [272]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns = ['EIN', 'NAME'], axis = 1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [273]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [274]:
# Dropping extra columns to see if the results change
# Drop the non-beneficial columns 'STATUS', 'ORGANIZATION', 'USE_CASE'
application_df = application_df.drop(columns = ['STATUS', 'ORGANIZATION', 'USE_CASE' ], axis = 1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,0,N,5000,1
1,T3,Independent,C2000,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,0,N,5000,0
3,T3,CompanySponsored,C2000,10000-24999,N,6692,1
4,T3,Independent,C1000,100000-499999,N,142590,1


In [275]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [276]:
# Look at APPLICATION_TYPE value counts for binning
application_vcs = application_df['APPLICATION_TYPE'].value_counts()
application_vcs

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [277]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(application_vcs[application_vcs < 528].index) 
cutoff_value = 528 

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [278]:
# Look at CLASSIFICATION value counts for binning
classification_vcs = application_df['CLASSIFICATION'].value_counts()
classification_vcs

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [279]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_vcs = classification_vcs[classification_vcs > 1]
classification_vcs

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [280]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_vcs[classification_vcs < 1500].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2235
C3000     1918
C2100     1883
C1370        1
C1236        1
C2190        1
C4200        1
C5200        1
C1248        1
C2600        1
C1580        1
C6100        1
C1820        1
C1900        1
C2570        1
C1570        1
C3700        1
C2500        1
C1283        1
C2380        1
C1732        1
C1728        1
C2170        1
C4120        1
C8210        1
C4500        1
C1245        1
C2561        1
C2150        1
Name: count, dtype: int64

In [281]:
# Convert categorical data to numeric with `pd.get_dummies`
numeric_data = pd.get_dummies(application_df)

In [282]:
# Split our preprocessed data into our features and target arrays
X = numeric_data.drop(['IS_SUCCESSFUL'], axis = 1)
y = numeric_data['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 58)

In [283]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [284]:
# Model 1

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 256
hidden_nodes_layer2 = 128
hidden_nodes_layer3 = 64
hidden_nodes_layer4 = 32


nn_model_1 = tf.keras.models.Sequential()

# Hidden layer1
nn_model_1.add(tf.keras.layers.Dense(units=hidden_nodes_layer1,
             input_dim=number_input_features, activation="relu")),tf.keras.layers.Dropout(0.5),  # Adding dropout for regularization

# Hidden layer2 - # Adding dropout for regularisation
nn_model_1.add(tf.keras.layers.Dense(
    units=hidden_nodes_layer2, activation="relu")),
tf.keras.layers.Dropout(0.5),  # Adding dropout for regularisation
    
# Hidden layer3
nn_model_1.add(tf.keras.layers.Dense(
    units=hidden_nodes_layer3, activation="relu"))
tf.keras.layers.Dropout(0.5), 
    
# Hidden layer4
nn_model_1.add(tf.keras.layers.Dense(
    units=hidden_nodes_layer4, activation="relu"))
tf.keras.layers.Dropout(0.5),  
    
# Output layer
nn_model_1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))


# Check the structure of the model
nn_model_1.summary()

Model: "sequential_33"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_157 (Dense)                    │ (None, 256)                 │          15,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_158 (Dense)                    │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_159 (Dense)                    │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_160 (Dense)                    │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_161 (Dense)                    │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 58,625 (229.00 KB)

 Trainable params: 58,625 (229.00 KB)

 Non-trainable params: 0 (0.00 B)

In [285]:
# Define early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)

In [286]:
# Compile the model
# nn_model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy']) choosing to use this adapted code below.
nn_model_1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [297]:
# Train the model
fit_model_1 = nn_model_1.fit(X_train_scaled, y_train, epochs = 20)

Epoch 1/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7323 - loss: 0.5551
Epoch 2/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7321 - loss: 0.5531
Epoch 3/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7337 - loss: 0.5502
Epoch 4/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7336 - loss: 0.5533
Epoch 5/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7322 - loss: 0.5526
Epoch 6/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7336 - loss: 0.5517
Epoch 7/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7284 - loss: 0.5573
Epoch 8/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7270 - loss: 0.5595
Epoch 9/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7329 - loss: 0.5541
Epoch 10/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7331 - loss: 0.5547
Epoch 11/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7290 - loss: 0.5594
Epoch 12/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step

In [298]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model_1.evaluate(X_test_scaled, y_test, verbose = 2)
print(f'Loss: {model_loss}, Accuracy: {model_accuracy}')

268/268 - 0s - 1ms/step - accuracy: 0.7235 - loss: 0.5681
Loss: 0.5681490302085876, Accuracy: 0.723498523235321


In [299]:
# Model 2:

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features_count = len(X_train_scaled[0])
hidden_nodes_layer1 = 12
hidden_nodes_layer2 = 6

nn_model_2 = tf.keras.models.Sequential()

# First hidden layer
nn_model_2.add(tf.keras.layers.Dense(units = hidden_nodes_layer1,
             input_dim = input_features_count, activation = 'tanh'))

# Second hidden layer
nn_model_2.add(tf.keras.layers.Dense(
    units = hidden_nodes_layer2, activation = 'relu'))

# Output layer
nn_model_2.add(tf.keras.layers.Dense(units = 1, activation= 'relu'))

# Check the structure of the model
nn_model_2.summary()

Model: "sequential_36"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_170 (Dense)                    │ (None, 12)                  │             720 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_171 (Dense)                    │ (None, 6)                   │              78 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_172 (Dense)                    │ (None, 1)                   │               7 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 805 (3.14 KB)

 Trainable params: 805 (3.14 KB)

 Non-trainable params: 0 (0.00 B)

In [300]:
# Compile model 2
nn_model_2.compile(loss='binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [301]:
# Fit model 2
fit_model_2 = nn_model_2.fit(X_train_scaled, y_train, epochs= 20)

Epoch 1/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.4991 - loss: 5.5026
Epoch 2/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.5764 - loss: 2.8503
Epoch 3/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6173 - loss: 2.1748
Epoch 4/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6301 - loss: 1.2638
Epoch 5/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6389 - loss: 1.1014
Epoch 6/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6537 - loss: 1.0507
Epoch 7/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6783 - loss: 1.0101
Epoch 8/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6777 - loss: 0.9898
Epoch 9/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6778 - loss: 0.9176
Epoch 10/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6915 - loss: 0.7212
Epoch 11/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6897 - loss: 0.6258
Epoch 12/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step

In [302]:
# Evaluate model 2 using the test data
model_loss, model_accuracy = nn_model_2.evaluate(X_test_scaled, y_test, verbose = 2)
print(f'Loss: {model_loss}, Accuracy: {model_accuracy}')

268/268 - 0s - 2ms/step - accuracy: 0.7217 - loss: 0.5793
Loss: 0.5793160796165466, Accuracy: 0.7217492461204529


In [303]:
# Model 3:

# Adding hidden layers
# Adding relu to replace tanh and using for accuracy

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features_count = len(X_train_scaled[0])
hidden_nodes_layer1 = 64
hidden_nodes_layer2 = 32
hidden_nodes_layer3 = 16
hidden_nodes_layer4 = 8

nn_model_3 = tf.keras.models.Sequential()

# Hidden layer1
nn_model_3.add(tf.keras.layers.Dense(units = hidden_nodes_layer1,
             input_dim = input_features_count, activation = 'relu'))

# Hidden layer2
nn_model_3.add(tf.keras.layers.Dense(
    units = hidden_nodes_layer2, activation = 'relu'))

# Hidden layer3
nn_model_3.add(tf.keras.layers.Dense(
    units = hidden_nodes_layer3, activation = 'relu'))

# Hidden layer4
nn_model_3.add(tf.keras.layers.Dense(
    units = hidden_nodes_layer4, activation = 'relu'))


# Output layer
nn_model_3.add(tf.keras.layers.Dense(units = 1, activation= 'sigmoid'))

# Check the structure of the model
nn_model_3.summary()

Model: "sequential_37"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_173 (Dense)                    │ (None, 64)                  │           3,840 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_174 (Dense)                    │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_175 (Dense)                    │ (None, 16)                  │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_176 (Dense)                    │ (None, 8)                   │             136 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_177 (Dense)                    │ (None, 1)                   │               9 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,593 (25.75 KB)

 Trainable params: 6,593 (25.75 KB)

 Non-trainable params: 0 (0.00 B)

In [304]:
# Compile the model
nn_model_3.compile(loss='binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [305]:
# Fit model 3
fit_model_3 = nn_model_3.fit(X_train_scaled, y_train, epochs= 20)

Epoch 1/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7020 - loss: 0.5979
Epoch 2/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7258 - loss: 0.5666
Epoch 3/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7264 - loss: 0.5644
Epoch 4/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7237 - loss: 0.5657
Epoch 5/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7265 - loss: 0.5632
Epoch 6/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7279 - loss: 0.5604
Epoch 7/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7253 - loss: 0.5655
Epoch 8/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7300 - loss: 0.5592
Epoch 9/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7295 - loss: 0.5580
Epoch 10/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7289 - loss: 0.5595
Epoch 11/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7271 - loss: 0.5638
Epoch 12/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step

In [306]:
# Evaluate model 2 using the test data
model_loss, model_accuracy = nn_model_3.evaluate(X_test_scaled, y_test, verbose = 2)
print(f'Loss: {model_loss}, Accuracy: {model_accuracy}')

268/268 - 1s - 2ms/step - accuracy: 0.7240 - loss: 0.5665
Loss: 0.5664882659912109, Accuracy: 0.7239649891853333


In [307]:
# Export our model to HDF5 file
# filepath = r'C:\Users\melbu\OneDrive\Documents\Class Activities\Assignments\Module 21\deep-learning-challenge/AlphabetSoupCharity_Optimisation.h5'
# nn_model.save(filepath)

nn_model.save('AlphabetSoupCharity_Optimisation.h5')